In [4]:
import pandas as pd
import numpy as np
import re
# from unidecode import unidecode
import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en_core_web_md', disable=["ner", "textcat", "entity_ruler", "merge_noun_chunks", "merge_entities", "merge_subtokens"])

In [5]:
from IPython.display import Image
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [6]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)

In [7]:
dfExoplanetsAbs=pd.read_json("./data/dfExoplanetsNASAabsClear_v1.json",orient="table")
dfExoplanetsSent=pd.read_json("./data/dfE.json",orient="table")

In [8]:
len(dfExoplanetsAbs)

31444

In [10]:
dfExoplanetsSent['annot']=dfExoplanetsSent.apply(lambda x: 0)
dfExoplanetsSent.head(3)

,IDabstr,IDsentence,sentence,annot
0,0,0,A new GEANT4 particle transport model - the Atmospheric Radiation Interaction Simulator - has been recently developed in order to model the interaction of radiation with planets.,NaN
1,0,1,"The upcoming instrumentational advancements in the exoplanetary science, in particular transit spectroscopy capabilities of missions like JWST and E-ELT, have motivated the development of a particle transport code with a focus on providing the necessary fl...",NaN
2,0,2,for the modeling of radiation environment for exoplanets.,NaN


In [12]:
sentences=list(dfExoplanetsSent['sentence'])
len(sentences)

251386

# Dependency label
https://github.com/clir/clearnlp-guidelines/blob/master/md/specifications/dependency_labels.md

https://stackoverflow.com/questions/40288323/what-do-spacys-part-of-speech-and-dependency-tags-mean/40288324

https://nlp.stanford.edu/software/dependencies_manual.pdf

In [13]:
satz=nlp(sentences[114522])
satz

We analyzed the Kepler light curves of four transiting hot Jupiter systems—KOI-13, HAT-P-7, TrES-2, and Kepler-76, which show BEaming, Ellipsoidal, and Reflection (BEER) phase modulations.

In [14]:
displacy.render(satz, style="dep")

In [15]:
[(t.text,t.pos_,t.dep_,t.head.text,t.head.pos_,[c for c in t.children]) for t in satz]

[('We', 'PRON', 'nsubj', 'analyzed', 'VERB', []),
 ('analyzed', 'VERB', 'ROOT', 'analyzed', 'VERB', [We, curves, .]),
 ('the', 'DET', 'det', 'curves', 'NOUN', []),
 ('Kepler', 'PROPN', 'nmod', 'curves', 'NOUN', []),
 ('light', 'NOUN', 'amod', 'curves', 'NOUN', []),
 ('curves',
  'NOUN',
  'dobj',
  'analyzed',
  'VERB',
  [the, Kepler, light, of, TrES-2]),
 ('of', 'ADP', 'prep', 'curves', 'NOUN', [systems]),
 ('four', 'NUM', 'nummod', 'systems', 'NOUN', []),
 ('transiting', 'VERB', 'amod', 'systems', 'NOUN', []),
 ('hot', 'ADJ', 'amod', 'Jupiter', 'PROPN', []),
 ('Jupiter', 'PROPN', 'compound', 'systems', 'NOUN', [hot]),
 ('systems', 'NOUN', 'pobj', 'of', 'ADP', [four, transiting, Jupiter, —]),
 ('—', 'PUNCT', 'punct', 'systems', 'NOUN', []),
 ('KOI-13', 'PROPN', 'nmod', 'P-7', 'NOUN', []),
 (',', 'PUNCT', 'punct', 'P-7', 'NOUN', []),
 ('HAT', 'PROPN', 'compound', 'P-7', 'NOUN', []),
 ('-', 'PUNCT', 'punct', 'P-7', 'NOUN', []),
 ('P-7', 'NOUN', 'punct', ',', 'PUNCT', [KOI-13, ,, HAT, -

In [16]:
def satz_analytic(satz):
#    merkmal = dict({"satz":satz.text})
    merkmal=dict({})
# search for main verb
    for t in satz:
        if t.dep_ == "ROOT":
            r=t.head.text
            merkmal.update({"act":r})
# subject and object related to verb
    for t in satz:
        if t.dep_ == "dobj" and merkmal["act"]==t.head.text:
            merkmal.update({"obj":''.join(w.text_with_ws for w in t.subtree)})
        if t.dep_ == "nsubj" and merkmal["act"]==t.head.text:
            merkmal.update({"subject":t.text.lower()})

    return(merkmal)
satz_analytic(satz)

{'act': 'analyzed',
 'subject': 'we',
 'obj': 'the Kepler light curves of four transiting hot Jupiter systems—KOI-13, HAT-P-7, TrES-2, and Kepler-76, which show BEaming, Ellipsoidal, and Reflection (BEER) phase modulations'}

In [17]:
df=pd.DataFrame(columns=['subject','act',"obj"])

In [18]:
#for i in range(len(sentences)):

for i in range(10):
    s=sentences[i]
    p=satz_analytic(nlp(s))
    df=df.append(p,ignore_index=True)

In [19]:
df=pd.read_pickle("/Users/gerdgrasshoff/Dropbox/temp/dfSentences.pickl")
#df=pd.read_pickle("c:/Dropbox/temp/dfSentences.pickl")

In [20]:
len(df)

251386

In [21]:
df.head(20)

,subject,act,obj
0,NaN,developed,NaN
1,advancements,motivated,the development of a particle transport code
2,NaN,for,NaN
3,flexibility,opens,possibilities for new studies
4,work,applies,AtRIS
5,we,benchmark,these first modeling results based on different GEANT4 physics lists with the energetic particle spectra recently measured by the Radiation Assessment Detector (RAD) on the surface of Mars
6,agreement,provides,one of the first and sound validations of AtRIS and the preferred physics list which could be recommended for predicting the radiation field of other conceivable
7,NaN,described,NaN
8,gr,is,NaN
9,p,V_(e,NaN


In [22]:
dfs=df[df["subject"]=="we"]
dfs.head(4)

,subject,act,obj
5,we,benchmark,these first modeling results based on different GEANT4 physics lists with the energetic particle spectra recently measured by the Radiation Assessment Detector (RAD) on the surface of Mars
11,we,perform,an in-depth literature search on average (av) and maximum (mx) wind velocity for each planet in the Solar system by various observational methods and altitudes
12,we,explore,a correlation between Ro and h expressed as a power law with index α based on wind velocities of planets in the Solar system
13,we,obtain,a correlation between Ro and h


In [23]:
len(dfs)

60946

In [24]:
dfAct=df.groupby(["subject","act"]).size().sort_values(ascending=False).reset_index()
dfAct[dfAct["subject"]=="we"]

,subject,act,0
0,we,present,7084
1,we,find,4935
2,we,show,2740
3,we,discuss,2121
4,we,report,2015
6,we,use,1784
7,we,propose,1390
8,we,describe,1357
9,we,investigate,1166
11,we,demonstrate,823


In [25]:
dfExoplanetsNASAannot = pd.read_json('./data/dfExoplanetsNASAdetected100rand_v2.json', orient = 'table')
del dfExoplanetsNASAannot['tagRootSent']
del dfExoplanetsNASAannot['tagDetected']
dfExoplanetsNASAannot.head(2)

,sent,label
0,"We detected visual companions within 1'' for 5 stars, between 1'' and 2'' for 7 stars, and between 2'' and 4'' for 15 stars.",discovery
1,"Using these data and photometry from the Spitzer Space Telescope, we have identified members with infrared excess emission from circumstellar disks and have estimated the evolutionary stages of the detected disks, which include 31 new full disks and 16 new...",discovery


In [26]:
def satz_analytic2(satz):
#    merkmal = dict({"satz":satz.text})
    merkmal=dict({})
# search for main verb
    for t in satz:
        if t.dep_ == "ROOT":
            r=t.head.text
            merkmal.update({"act":r})
# subject and object related to verb
    for t in satz:
#        if t.dep_ == "dobj" and merkmal["act"]==t.head.text:
#            merkmal.update({"obj":''.join(w.text_with_ws for w in t.subtree)})
        if t.dep_ == "nsubj" and merkmal["act"]==t.head.text:
            merkmal.update({"subject":t.text.lower()})

    return(merkmal)
satz_analytic(satz)

{'act': 'analyzed',
 'subject': 'we',
 'obj': 'the Kepler light curves of four transiting hot Jupiter systems—KOI-13, HAT-P-7, TrES-2, and Kepler-76, which show BEaming, Ellipsoidal, and Reflection (BEER) phase modulations'}

In [27]:
dfExoplanetsNASAannot["s"]=dfExoplanetsNASAannot["sent"].apply(lambda y: satz_analytic2(nlp(y)))
dfExoplanetsNASAannot.head(2)

,sent,label,s
0,"We detected visual companions within 1'' for 5 stars, between 1'' and 2'' for 7 stars, and between 2'' and 4'' for 15 stars.",discovery,"{'act': 'detected', 'subject': 'we'}"
1,"Using these data and photometry from the Spitzer Space Telescope, we have identified members with infrared excess emission from circumstellar disks and have estimated the evolutionary stages of the detected disks, which include 31 new full disks and 16 new...",discovery,"{'act': 'identified', 'subject': 'we'}"


In [28]:
def transp(x):
    if x=="discovery":
        y=1
    else:
        y=0
    return(y)
dfExoplanetsNASAannot["label"]=dfExoplanetsNASAannot["label"].apply(lambda x: transp(x))

In [29]:
dfExoplanetsNASAannot.head(2)

,sent,label,s
0,"We detected visual companions within 1'' for 5 stars, between 1'' and 2'' for 7 stars, and between 2'' and 4'' for 15 stars.",1,"{'act': 'detected', 'subject': 'we'}"
1,"Using these data and photometry from the Spitzer Space Telescope, we have identified members with infrared excess emission from circumstellar disks and have estimated the evolutionary stages of the detected disks, which include 31 new full disks and 16 new...",1,"{'act': 'identified', 'subject': 'we'}"


In [30]:
_all_xs=list(dfExoplanetsNASAannot["s"])

In [31]:
_all_xs[1]

{'act': 'identified', 'subject': 'we'}

In [32]:
_all_ys=np.array(list(dfExoplanetsNASAannot["label"]))
_all_ys

array([1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1])

In [33]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

all_xs, all_ys = shuffle(_all_xs, _all_ys, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(
    all_xs, all_ys, test_size=0.25, random_state=0)
print('{} items total, {:.1%} true'.format(len(all_xs), np.mean(all_ys)))

100 items total, 40.0% true


In [ ]:
from xgboost import XGBClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

clf = XGBClassifier()
vec = DictVectorizer()
pipeline = make_pipeline(vec, clf)

def evaluate(_clf):
    scores = cross_val_score(_clf, all_xs, all_ys, scoring='accuracy', cv=10)
    print('Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    _clf.fit(X_train, y_train)  # so that parts of the original pipeline are fitted
    
evaluate(pipeline)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
booster = clf.get_booster()
original_feature_names = booster.feature_names
booster.feature_names = vec.get_feature_names()
print(booster.get_dump()[0])
# recover original feature names
booster.feature_names = original_feature_names

In [ ]:
from eli5 import show_weights
show_weights(clf, vec=vec)

In [ ]:
@interact
def show_abstracts(x=(1,29000,1)):
    df=dfExoplanetsSent.loc[dfExoplanetsSent["IDabstr"] == x]
    return(df)